# Kinetics from TST

## Example for $F + H_2 \rightleftharpoons H + HF$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline


import rmgpy.data.kinetics
import rmgpy.reaction

In [ ]:
# load the RMG database
ref_lib = '2-BTP/full'
ref_library_path = "/home/harris.se/rmg/RMG-database/input/kinetics/"
ref_database = rmgpy.data.kinetics.KineticsDatabase()
ref_database.load(
    ref_library_path,
    libraries=[
        ref_lib,
    ],
    families=[],
)

In [ ]:
# Find the reaction label
ref_lib = '2-BTP/full'
reaction_label = 'H2 + F <=> H + HF'
for key in ref_database.libraries[ref_lib].entries.keys():
#     print(ref_database.libraries[ref_lib].entries[key].label)
    if ref_database.libraries[ref_lib].entries[key].label == reaction_label:
        print(ref_database.libraries[ref_lib].entries[key].label)
        rmg_kinetics = ref_database.libraries[ref_lib].entries[key].data
        break

In [ ]:
import rmgpy.quantity
import rmgpy.constants

In [ ]:
rmgpy.constants.kB

In [ ]:
# Compare to experimental values from here: https://srd.nist.gov/jpcrdreprint/1.555692.pdf
exp_kinetics = rmgpy.kinetics.arrhenius.Arrhenius()
exp_kinetics.A = rmgpy.quantity.RateCoefficient(2.7e12, 'cm^3/(mol*s)')
exp_kinetics.n = 0.5
exp_kinetics.Ea = rmgpy.quantity.Energy(rmgpy.constants.R * 0.319, 'kJ/mol')

In [ ]:
%matplotlib inline
# Make the Arrhenius plot using RMG
print(ref_database.libraries[ref_lib].entries[key].label)
ref_database.libraries[ref_lib].entries[key].data

plt.xlabel('1/T ($K^{-1})$')
plt.ylabel('ln(k)')
plt.title('Reaction Rate vs. Inverse Temperature')
T_ = np.linspace(300, 3000, 1001)
T_inv = 1.0 / T_

k_rmg = np.zeros(len(T_))
k_exp = np.zeros(len(T_))
for i in range(0, len(T_)):
    k_rmg[i] = rmg_kinetics.get_rate_coefficient(T_[i])
    k_exp[i] = exp_kinetics.get_rate_coefficient(T_[i])
plt.plot(T_inv, np.log(k_rmg), T_inv, np.log(k_exp))
plt.legend(['RMG', 'Cohen Westberg'])